In [1]:
import json
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
#pd.read_json('C:/src/github.com/ongzhixian/mini-trade/sample-data/ohlc/xau_usd.json')
#data = pd.read_json('./xau_usd.json')

with open('./xau_usd.json') as in_file:
    data = json.load(in_file)
 
#instruments_df = pd.DataFrame(data['instruments'])

# candles = pd.DataFrame(data['candles'])
# #candles[ ['time', 'volume', 'complete', 'mid'] ]
# #candles.values.flatten()
# candles


In [3]:
# data
# extracted_data = []
# for index, row in data.condles:
#     extracted_data.append({
#         "volume": row['volume'],
#         "time": row['time'],
#         "complete": row['complete'],
#         "o": row['mid']['o'],
#         "h": row['mid']['h'],
#         "l": row['mid']['l'],
#         "c": row['mid']['c']
#     })

In [4]:
extracted_data = []

for row in data['candles']:
    extracted_data.append({
        "time": row['time'],
        "complete": row['complete'],
        "Open": pd.to_numeric(row['mid']['o']),
        "High": pd.to_numeric(row['mid']['h']),
        "Low": pd.to_numeric(row['mid']['l']),
        "Close": pd.to_numeric(row['mid']['c']),
        "Volume": row['volume']
    })


In [5]:
pd.DataFrame(extracted_data)

pd.DataFrame(extracted_data).info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   time      500 non-null    object 
 1   complete  500 non-null    bool   
 2   Open      500 non-null    float64
 3   High      500 non-null    float64
 4   Low       500 non-null    float64
 5   Close     500 non-null    float64
 6   Volume    500 non-null    int64  
dtypes: bool(1), float64(4), int64(1), object(1)
memory usage: 24.1+ KB


In [6]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

def generate_trading_signals_with_forecast(df, short_window=20, long_window=50, rsi_period=14, overbought=70, oversold=30, bollinger_window=20, bollinger_std=2, forecast_window=10):
    """
    Generates advanced trading signals with direction, strength, risk level, and forecast price prediction.

    Args:
        df (pd.DataFrame): DataFrame with OHLCV data.
        short_window (int): Short-term moving average window.
        long_window (int): Long-term moving average window.
        rsi_period (int): Period for RSI calculation.
        overbought (int): Overbought level.
        oversold (int): Oversold level.
        bollinger_window (int): Window for Bollinger Bands.
        bollinger_std (int): Number of standard deviations for Bollinger Bands.
        forecast_window (int): Window for price prediction.

    Returns:
        pd.DataFrame: DataFrame with advanced trading signals and forecast.
    """

    # Moving Averages
    df['short_ma'] = df['Close'].rolling(window=short_window).mean()
    df['long_ma'] = df['Close'].rolling(window=long_window).mean()
    df['ma_signal'] = np.where(df['short_ma'] > df['long_ma'], 1, np.where(df['short_ma'] < df['long_ma'], -1, 0))
    df['ma_strength'] = (abs(df['short_ma'] - df['long_ma']) / df['Close']) * 100

    # RSI
    delta = df['Close'].diff()
    up = delta.clip(lower=0)
    down = -1 * delta.clip(upper=0)
    ema_up = up.rolling(window=rsi_period).mean()
    ema_down = down.rolling(window=rsi_period).mean()
    rs = ema_up / ema_down
    df['rsi'] = 100 - (100 / (1 + rs))
    df['rsi_signal'] = np.where(df['rsi'] < oversold, 1, np.where(df['rsi'] > overbought, -1, 0))
    df['rsi_strength'] = np.where(df['rsi_signal'] == 1, (oversold - df['rsi']), np.where(df['rsi_signal'] == -1, (df['rsi'] - overbought), 0))

    # Bollinger Bands
    df['middle_band'] = df['Close'].rolling(window=bollinger_window).mean()
    df['std_dev'] = df['Close'].rolling(window=bollinger_window).std()
    df['upper_band'] = df['middle_band'] + (df['std_dev'] * bollinger_std)
    df['lower_band'] = df['middle_band'] - (df['std_dev'] * bollinger_std)
    df['bollinger_signal'] = np.where(df['Close'] < df['lower_band'], 1, np.where(df['Close'] > df['upper_band'], -1, 0))
    df['bollinger_strength'] = np.where(df['bollinger_signal'] == 1, ((df['lower_band'] - df['Close']) / df['Close']) * 100, np.where(df['bollinger_signal'] == -1, ((df['Close'] - df['upper_band']) / df['Close']) * 100, 0))

    # Combined Signal and Risk
    df['combined_signal'] = np.sign(df['ma_signal'] + df['rsi_signal'] + df['bollinger_signal'])
    df['combined_strength'] = (df['ma_strength'] + df['rsi_strength'] + df['bollinger_strength']).abs() / 3
    df['risk_level'] = (100 - df['combined_strength']).clip(0, 100)
    df['position'] = df['combined_signal'].diff()

    # Price Prediction (Linear Regression)
    forecast_prices = []
    for i in range(forecast_window, len(df)):
        train_data = df['Close'][i - forecast_window:i].values.reshape(-1, 1)
        train_index = np.arange(0, forecast_window).reshape(-1, 1)
        model = LinearRegression()
        model.fit(train_index, train_data)
        forecast_price = model.predict([[forecast_window]])[0][0]
        forecast_prices.append(forecast_price)

    df['forecast_price'] = np.nan
    df.loc[forecast_window:, 'forecast_price'] = forecast_prices

    return df

# Example usage:
# Assuming you have a DataFrame called 'ohlcv_df' with columns 'Open', 'High', 'Low', 'Close', 'Volume'
# ohlcv_df = ... (load your data)
# ohlcv_df = generate_trading_signals_with_forecast(ohlcv_df)
# print(ohlcv_df[['Close', 'combined_signal', 'combined_strength', 'risk_level', 'forecast_price', 'position']].tail(20))

In [7]:
ohlcv_df = pd.DataFrame(extracted_data)
ohlcv_df = generate_trading_signals_with_forecast(ohlcv_df)
print(ohlcv_df[['Close', 'combined_signal', 'combined_strength', 'risk_level', 'forecast_price', 'position']].tail(20))

        Close  combined_signal  combined_strength  risk_level  forecast_price  \
480  3034.905                1           0.002518   99.997482     3034.772667   
481  3034.925                1           0.002207   99.997793     3034.792000   
482  3034.980                1           0.002017   99.997983     3034.837333   
483  3034.990                1           0.001908   99.998092     3034.909000   
484  3034.745                1           0.001628   99.998372     3034.967000   
485  3034.820                1           0.001328   99.998672     3034.902667   
486  3034.615                1           0.001012   99.998988     3034.916333   
487  3034.820                1           0.000711   99.999289     3034.802000   
488  3034.150                1           0.003356   99.996644     3034.775333   
489  3034.125                1           0.767359   99.232641     3034.486333   
490  3034.095                0           0.001484   99.998516     3034.231000   
491  3034.195               

In [8]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA

def generate_trading_signals_with_arima_forecast(df, short_window=20, long_window=50, rsi_period=14, overbought=70, oversold=30, bollinger_window=20, bollinger_std=2, arima_order=(5, 1, 0), forecast_window=10):
    """
    Generates advanced trading signals with direction, strength, risk level, and ARIMA forecast price prediction.

    Args:
        df (pd.DataFrame): DataFrame with OHLCV data.
        short_window (int): Short-term moving average window.
        long_window (int): Long-term moving average window.
        rsi_period (int): Period for RSI calculation.
        overbought (int): Overbought level.
        oversold (int): Oversold level.
        bollinger_window (int): Window for Bollinger Bands.
        bollinger_std (int): Number of standard deviations for Bollinger Bands.
        arima_order (tuple): ARIMA model order (p, d, q).
        forecast_window (int): Window for price prediction.

    Returns:
        pd.DataFrame: DataFrame with advanced trading signals and forecast.
    """

    # Moving Averages
    df['short_ma'] = df['Close'].rolling(window=short_window).mean()
    df['long_ma'] = df['Close'].rolling(window=long_window).mean()
    df['ma_signal'] = np.where(df['short_ma'] > df['long_ma'], 1, np.where(df['short_ma'] < df['long_ma'], -1, 0))
    df['ma_strength'] = (abs(df['short_ma'] - df['long_ma']) / df['Close']) * 100

    # RSI
    delta = df['Close'].diff()
    up = delta.clip(lower=0)
    down = -1 * delta.clip(upper=0)
    ema_up = up.rolling(window=rsi_period).mean()
    ema_down = down.rolling(window=rsi_period).mean()
    rs = ema_up / ema_down
    df['rsi'] = 100 - (100 / (1 + rs))
    df['rsi_signal'] = np.where(df['rsi'] < oversold, 1, np.where(df['rsi'] > overbought, -1, 0))
    df['rsi_strength'] = np.where(df['rsi_signal'] == 1, (oversold - df['rsi']), np.where(df['rsi_signal'] == -1, (df['rsi'] - overbought), 0))

    # Bollinger Bands
    df['middle_band'] = df['Close'].rolling(window=bollinger_window).mean()
    df['std_dev'] = df['Close'].rolling(window=bollinger_window).std()
    df['upper_band'] = df['middle_band'] + (df['std_dev'] * bollinger_std)
    df['lower_band'] = df['middle_band'] - (df['std_dev'] * bollinger_std)
    df['bollinger_signal'] = np.where(df['Close'] < df['lower_band'], 1, np.where(df['Close'] > df['upper_band'], -1, 0))
    df['bollinger_strength'] = np.where(df['bollinger_signal'] == 1, ((df['lower_band'] - df['Close']) / df['Close']) * 100, np.where(df['bollinger_signal'] == -1, ((df['Close'] - df['upper_band']) / df['Close']) * 100, 0))

    # Combined Signal and Risk
    df['combined_signal'] = np.sign(df['ma_signal'] + df['rsi_signal'] + df['bollinger_signal'])
    df['combined_strength'] = (df['ma_strength'] + df['rsi_strength'] + df['bollinger_strength']).abs() / 3
    df['risk_level'] = (100 - df['combined_strength']).clip(0, 100)
    df['position'] = df['combined_signal'].diff()

    # Price Prediction (ARIMA)
    forecast_prices = []
    for i in range(forecast_window, len(df)):
        train_data = df['Close'][i - forecast_window:i].values
        try:
            model = ARIMA(train_data, order=arima_order)
            model_fit = model.fit()
            forecast = model_fit.forecast(steps=1)[0]
            forecast_prices.append(forecast)
        except Exception as e:
            print(f"ARIMA error at index {i}: {e}")
            forecast_prices.append(np.nan) #Append NaN if ARIMA fails.

    df['forecast_price'] = np.nan
    df.loc[forecast_window:, 'forecast_price'] = forecast_prices

    return df

# Example usage:
# Assuming you have a DataFrame called 'ohlcv_df' with columns 'Open', 'High', 'Low', 'Close', 'Volume'
# ohlcv_df = ... (load your data)
# ohlcv_df = generate_trading_signals_with_arima_forecast(ohlcv_df)
# print(ohlcv_df[['Close', 'combined_signal', 'combined_strength', 'risk_level', 'forecast_price', 'position']].tail(20))

In [9]:
ohlcv_df = generate_trading_signals_with_arima_forecast(ohlcv_df)
print(ohlcv_df[['Close', 'combined_signal', 'combined_strength', 'risk_level', 'forecast_price', 'position']].tail(20))

c:\Apps\Python\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Apps\Python\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Apps\Python\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Apps\Python\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Apps\Python\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimizatio

ARIMA error at index 492: LU decomposition error.


c:\Apps\Python\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


        Close  combined_signal  combined_strength  risk_level  forecast_price  \
480  3034.905                1           0.002518   99.997482     3034.790000   
481  3034.925                1           0.002207   99.997793     3034.688592   
482  3034.980                1           0.002017   99.997983     3034.925000   
483  3034.990                1           0.001908   99.998092     3034.917104   
484  3034.745                1           0.001628   99.998372     3034.905028   
485  3034.820                1           0.001328   99.998672     3034.745000   
486  3034.615                1           0.001012   99.998988     3034.680534   
487  3034.820                1           0.000711   99.999289     3034.741726   
488  3034.150                1           0.003356   99.996644     3034.849548   
489  3034.125                1           0.767359   99.232641     3034.509043   
490  3034.095                0           0.001484   99.998516     3033.806528   
491  3034.195               

c:\Apps\Python\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
